GÖREV 1 -SENTETİK VERİ ÜRETME-

In [2]:
"""
Düz -> Z: +9.8
Ters -> z: -9.8
Sağa Yatık -> Y: +9.8
Sola Yatık -> Y: -9.8
Arkaya Yatık -> X: +9.8
Öne Yatık -> X: -9.8
"""

import numpy as np
import pandas as pd
import tensorflow as tf

#Random state'i kitleme
np.random.seed(42)
tf.random.set_seed(42)
dataNum = 100
noise = 0.3

#DÜZLER
duzX = np.random.normal(0.0, scale=noise, size=dataNum)
duzY = np.random.normal(0.0, scale=noise, size=dataNum)
duzZ = np.random.normal(9.8, scale=noise, size=dataNum)

#TERSLER
tersX = np.random.normal(0.0, scale=noise, size=dataNum)
tersY = np.random.normal(0.0, scale=noise, size=dataNum)
tersZ = np.random.normal(-9.8, scale=noise, size=dataNum)

#SAĞA YATIKLAR
sağX = np.random.normal(0.0, scale=noise, size=dataNum)
sağY = np.random.normal(9.8, scale=noise, size=dataNum)
sağZ = np.random.normal(0.0, scale=noise, size=dataNum)

#SOLA YATIKLAR
solX = np.random.normal(0.0, scale=noise, size=dataNum)
solY = np.random.normal(-9.8, scale=noise, size=dataNum)
solZ = np.random.normal(0.0, scale=noise, size=dataNum)

#ARKAYA YATIK
arkaX = np.random.normal(9.8, scale=noise, size=dataNum)
arkaY = np.random.normal(0.0, scale=noise, size=dataNum)
arkaZ = np.random.normal(0.0, scale=noise, size=dataNum)

#ÖNE YATIK
önX = np.random.normal(-9.8, scale=noise, size=dataNum)
önY = np.random.normal(0.0, scale=noise, size=dataNum)
önZ = np.random.normal(0.0, scale=noise, size=dataNum)

duzData = np.column_stack([duzX, duzY, duzZ])
tersData = np.column_stack([tersX, tersY, tersZ])

sağData = np.column_stack([sağX, sağY, sağZ])
solData = np.column_stack([solX, solY, solZ])

arkaData = np.column_stack([arkaX, arkaY, arkaZ])
önData = np.column_stack([önX, önY, önZ])

totalData = np.vstack([duzData, tersData, sağData, solData, arkaData, önData])

stateDuz = np.zeros(dataNum)
stateTers = np.ones(dataNum)
stateSağ = np.ones(dataNum)*2
stateSol = np.ones(dataNum)*3
stateArka = np.ones(dataNum)*4
stateÖn = np.ones(dataNum)*5

statesTotal = np.hstack([stateDuz, stateTers, stateSağ, stateSol, stateArka, stateÖn])

df = pd.DataFrame(data={
    "ivmeX": totalData[:, 0],
    "ivmeY": totalData[:, 1],
    "ivmeZ": totalData[:, 2],
    "durum": statesTotal

})
df = df.astype({
    "ivmeX": "float32",
    "ivmeY": "float32",
    "ivmeZ": "float32",
    "durum": "int8"
})

print(df.memory_usage(deep=True).sum())
print(df["durum"].value_counts())


7932
durum
0    100
1    100
2    100
3    100
4    100
5    100
Name: count, dtype: int64


GÖREV2 -VERİYİ ML'E HAZIR HÂLE GETİRME-

In [6]:
from sklearn.model_selection import train_test_split
X = df[["ivmeX","ivmeY","ivmeZ"]]
Y = df["durum"]
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)


GÖREV 3 -VERİLERİ ORTAK ARALIĞA AL VE MODELİ OLUŞTUR-

In [4]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
scaler = StandardScaler()

scaler.fit(xtrain)
scaledxTrain = scaler.transform(xtrain)
scaledxTest = scaler.transform(xtest)


model = Sequential()
model.add(tf.keras.Input(shape=(3,))) #Keras 3.0 Özelliği ****
model.add(Dense(12, activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(6, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()

scaledDf = pd.DataFrame(scaledxTrain, columns=xtrain.columns)

model.fit(scaledDf, ytrain, epochs=30, batch_size=10, validation_data=(scaledxTest, ytest))

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 12)             │            48 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 6)              │            54 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 206 (824.00 B)

 Trainable params: 206 (824.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1667 - loss: 1.6240 - val_accuracy: 0.2917 - val_loss: 1.5224
Epoch 2/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3292 - loss: 1.4334 - val_accuracy: 0.3333 - val_loss: 1.3438
Epoch 3/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4521 - loss: 1.2624 - val_accuracy: 0.6917 - val_loss: 1.1715
Epoch 4/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8083 - loss: 1.0888 - val_accuracy: 0.8333 - val_loss: 0.9976
Epoch 5/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8333 - loss: 0.9191 - val_accuracy: 0.8333 - val_loss: 0.8306
Epoch 6/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8333 - loss: 0.7562 - val_accuracy: 0.8333 - val_loss: 0.6717
Epoch 7/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8333 - loss: 0.5986 - val_accuracy: 0.8333 - val_loss: 0.5123
Epoch 8/30
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8583 - loss: 0.4519 - val_accuracy: 0.9750 - val_loss:

GÖREV 3 -TESTİ BİTİR VE EN ZOR SORUYU VER-

In [5]:
loss, accuracy = model.evaluate(scaledxTest, ytest)
print(f"Test Kaybı (Loss): {loss:.4f}")
print(f"Test Başarısı (Accuracy): {accuracy * 100:.2f}%")

newData = np.array([[0.0, 5.0, 5.0]]) + noise
scalednewData = scaler.transform(newData)
prediction = model.predict(scalednewData)
print(prediction)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0026  
Test Kaybı (Loss): 0.0026
Test Başarısı (Accuracy): 100.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
[[9.9148554e-01 2.2058027e-04 9.1256812e-04 8.8138891e-05 5.0132819e-03
  2.2798879e-03]]


c:\Users\User\Desktop\yazilim\python\veri bilimi\gemini\dersler\DEMO İHA PROJECTS\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
